# 第二章 文档加载

 - [一、PDF文档](#一、PDF文档)
     - [1.1 安装相关包 ](#1.1-安装相关包-)
     - [1.2 加载PDF文档](#1.2-加载PDF文档)
     - [1.3 探索加载的数据](#1.3-探索加载的数据)
 - [二、YouTube音频](#二、YouTube音频)
     - [2.1 安装相关包 ](#2.1-安装相关包-)
     - [2.2 加载Youtube音频文档](#2.2-加载Youtube音频文档)
     - [2.3 探索加载的数据](#2.3-探索加载的数据)
 - [三、网页文档](#三、网页文档)
     - [3.1 加载网页文档](#3.1-加载网页文档)
     - [3.2 探索加载的数据](#3.2-探索加载的数据)
 - [四、Notion文档](#四、Notion文档)
     - [4.1 加载Notion Markdown文档](#4.1-加载Notion-Markdown文档)
     - [4.2 探索加载的数据](#4.2-探索加载的数据)


同前序课程一样，我们同样需要先配置环境以访问 OpenAI 提供的 API，配置方法如下，详细介绍请参见其他课程

In [1]:
# 下载最新版 LangChain
!pip install -q langchain --upgrade

In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# openai.api_key  = os.environ['OPENAI_API_KEY']

## 一、PDF文档

首先，我们来加载一个[PDF文档](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf)。该文档为吴恩达教授的2009年机器学习课程的字幕文件。因为这些字幕为自动生成，所以词句直接可能不太连贯和通畅。

### 1.1 安装相关包 

In [1]:
!pip install -q pypdf

### 1.2 加载PDF文档

In [3]:
from langchain.document_loaders import PyPDFLoader

# 创建一个 PyPDFLoader Class 实例，输入为待加载的pdf文档路径
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")

# 调用 PyPDFLoader Class 的函数 load对pdf文件进行加载
pages = loader.load()

### 1.3 探索加载的数据

文档加载后储存在`pages`变量中:
- `page`的变量类型为`List`
- 打印 `pages` 的长度可以看到pdf一共包含多少页

In [4]:
print(type(pages))

<class 'list'>


In [5]:
print(len(pages))

22


`page`中的每一元素为一个文档，变量类型为`langchain.schema.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [6]:
page = pages[0]

In [7]:
print(type(page))

<class 'langchain_core.documents.base.Document'>


In [8]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning i


In [9]:
print(page.metadata)

{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}


## 二、YouTube音频

在第一部分的内容，我们学习了如何加载PDF文档。在这部分的内容，我们学习对于给定的 YouTube 视频链接
- 如何使用LangChain加载器将视频的音频下载到本地
- 然后使用OpenAIWhisperPaser解析器将音频转化为文本

### 2.1 安装相关包 

In [9]:
!pip -q install yt_dlp
!pip -q install pydub

### 2.2 加载Youtube音频文档

In [10]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [11]:
url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir="docs/youtube/"

# 创建一个 GenericLoader Class 实例
loader = GenericLoader(
    #将链接url中的Youtube视频的音频下载下来,存在本地路径save_dir
    YoutubeAudioLoader([url],save_dir), 
    
    #使用OpenAIWhisperPaser解析器将音频转化为文本
    OpenAIWhisperParser()
)

# 调用 GenericLoader Class 的函数 load对视频的音频文件进行加载
docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=jGwO_UgTS7I
[youtube] jGwO_UgTS7I: Downloading webpage
[youtube] jGwO_UgTS7I: Downloading ios player API JSON
[youtube] jGwO_UgTS7I: Downloading player 84314bef
[youtube] jGwO_UgTS7I: Downloading m3u8 information
[info] jGwO_UgTS7I: Downloading 1 format(s): 140
[download] docs/youtube//Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a has already been downloaded
[download] 100% of   69.71MiB


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


DownloadError: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location

### 2.3 探索加载的数据

文档加载后储存在`docs`变量中:
- `docs`的变量类型为`List`
- 打印 `docs` 的长度可以看到一共包含多少页

In [15]:
print(type(docs))

<class 'list'>


In [16]:
print(len(docs))

1


`docs`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [19]:
doc = docs[0]

In [20]:
print(type(doc))

<class 'langchain.schema.document.Document'>


In [13]:
print(doc.page_content[0:500])

Welcome to CS229 Machine Learning. Uh, some of you know that this is a class that's taught at Stanford for a long time. And this is often the class that, um, I most look forward to teaching each year because this is where we've helped, I think, several generations of Stanford students become experts in machine learning, got- built many of their products and services and startups that I'm sure, many of you or probably all of you are using, uh, uh, today. Um, so what I want to do today was spend s


In [14]:
print(doc.metadata)

{'source': 'docs/youtube/Stanford CS229： Machine Learning Course, Lecture 1 - Andrew Ng (Autumn 2018).m4a', 'chunk': 0}


## 三、网页文档

在第二部分，我们对于给定的 YouTube 视频链接 (URL)，使用 LangChain 加载器将视频的音频下载到本地，然后使用 OpenAIWhisperPaser 解析器将音频转化为文本。

本部分，对于给定网页文档链接(URLs)，我们学习如何对其进行加载。这里我们对Github上的网页文档进行加载，该文档格式为markdown。

### 3.1 加载网页文档

In [21]:
from langchain.document_loaders import WebBaseLoader

# 创建一个 WebBaseLoader Class 实例
url = "https://github.com/basecamp/handbook/blob/master/37signals-is-you.md"
header = {'User-Agent': 'python-requests/2.27.1', 
          'Accept-Encoding': 'gzip, deflate, br', 
          'Accept': '*/*',
          'Connection': 'keep-alive'}
loader = WebBaseLoader(web_path=url,header_template=header)

# 调用 WebBaseLoader Class 的函数 load对文件进行加载
docs = loader.load()

### 3.2 探索加载的数据

文档加载后储存在`docs`变量中:
- `docs`的变量类型为`List`
- 打印 `docs` 的长度可以看到一共包含多少页

In [22]:
print(type(docs))

<class 'list'>


In [23]:
print(len(docs))

1


`docs`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [24]:
doc = docs[0]

In [25]:
print(type(doc))

<class 'langchain.schema.document.Document'>


In [26]:
print(doc.page_content)

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"37signals-is-you.md","path":"37signals-is-you.md","contentType":"file"},{"name":"LICENSE.md","path":"LICENSE.md","contentType":"file"},{"name":"README.md","path":"README.md","contentType":"file"},{"name":"benefits-and-perks.md","path":"benefits-and-perks.md","contentType":"file"},{"name":"code-of-conduct.md","path":"code-of-conduct.md","contentType":"file"},{"name":"faq.md","path":"faq.md","contentType":"file"},{"name":"getting-started.md","path":"getting-started.md","contentType":"file"},{"name":"how-we-work.md","path":"how-we-work.md","contentType":"file"},{"name":"international-travel-guide.md","path":"international-travel-guide.md","contentType":"file"},{"name":"making-a-career.md","path":"making-a-career.md","contentType":"file"},{"name":"managing-work-devices.md","path":"managing-work-devices.md","contentType":"file"},{"name":"moonlighting.md","path":"moonlighting.md","contentType":"file"},{"name":"our-inte

可以看到上面的文档内容包含许多冗余的信息。通常来讲，我们需要进行对这种数据进行进一步处理(Post Processing)。

In [27]:
import json
convert_to_json = json.loads(doc.page_content)
extracted_markdow = convert_to_json['payload']['blob']['richText']
print(extracted_markdow)

37signals Is You
Everyone working at 37signals represents 37signals. When a customer gets a response from Merissa on support, Merissa is 37signals. When a customer reads a tweet by Eron that our systems are down, Eron is 37signals. In those situations, all the other stuff we do to cultivate our best image is secondary. What’s right in front of someone in a time of need is what they’ll remember.
That’s what we mean when we say marketing is everyone’s responsibility, and that it pays to spend the time to recognize that. This means avoiding the bullshit of outage language and bending our policies, not just lending your ears. It means taking the time to get the writing right and consider how you’d feel if you were on the other side of the interaction.
The vast majority of our customers come from word of mouth and much of that word comes from people in our audience. This is an audience we’ve been educating and entertaining for 20 years and counting, and your voice is part of us now, whether

## 四、Notion文档

- 点击[Notion示例文档](https://yolospace.notion.site/Blendle-s-Employee-Handbook-e31bff7da17346ee99f531087d8b133f)右上方复制按钮(Duplicate)，复制文档到你的Notion空间
- 点击右上方`⋯` 按钮，选择导出为Mardown&CSV。导出的文件将为zip文件夹
- 解压并保存mardown文档到本地路径`docs/Notion_DB/`

### 4.1 加载Notion Markdown文档

In [28]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()

### 4.2 探索加载的数据

文档加载后储存在`docs`变量中:
- `docs`的变量类型为`List`
- 打印 `docs` 的长度可以看到一共包含多少页

In [29]:
print(type(docs))

<class 'list'>


In [30]:
print(len(docs))

1


`docs`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [31]:
doc = docs[0]

In [32]:
print(type(doc))

<class 'langchain.schema.document.Document'>


In [33]:
print(doc.page_content[0:500])

# Blendle's Employee Handbook

This is a living document with everything we've learned working with people while running a startup. And, of course, we continue to learn. Therefore it's a document that will continue to change. 

**Everything related to working at Blendle and the people of Blendle, made public.**

These are the lessons from three years of working with the people of Blendle. It contains everything from [how our leaders lead](https://www.notion.so/ecfb7e647136468a9a0a32f1771a8f52?pv
